In [14]:
# loading data from NSRDB (https://nsrdb.nrel.gov/data-viewer)

import pandas as pd
import numpy as np

In [15]:
# elevation angle is 90 - (solar zenith angle)
# estimate global tilted irradiance by:
# GTI = direct normal irradiance (DNI) * sin(elevation + panel tilt) + diffuse horizontal irradiance (DHI)

# panels are rated according to peak solar hours (PSH), which corresponds to a solar irradiance of 1000 W/m^2
# if we compute GTI, we can estimate the percentage of this peak output that a panel will produce (minus 14% system losses)

df = pd.read_csv('caltech_solar_radiation_2021.csv', parse_dates=True)

df['datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute']])
df.drop(['Year', 'Month', 'Day', 'Hour', 'Minute'], axis=1, inplace=True)
df.set_index('datetime', inplace=True)

df['Solar Elevation Angle'] = 90 - df['Solar Zenith Angle']

PANEL_TILT = 34.5 # degrees

df['GTI'] = df['DNI'] * np.sin(np.radians(df['Solar Elevation Angle'])) + df['DHI']

display(df)

,DHI,DNI,Dew Point,Surface Albedo,Wind Speed,Relative Humidity,Temperature,Pressure,GHI,Solar Zenith Angle,Solar Elevation Angle,GTI
datetime,,,,,,,,,,,,
2021-01-01 00:30:00,0,0,-0.8,0.13,3.7,55.86,7.5,969,0,166.59,-76.59,0.0
2021-01-01 01:30:00,0,0,-1.6,0.13,3.4,53.24,7.3,970,0,156.65,-66.65,0.0
2021-01-01 02:30:00,0,0,-2.0,0.13,3.4,52.12,7.2,970,0,144.67,-54.67,0.0
2021-01-01 03:30:00,0,0,-2.2,0.13,3.4,51.76,7.1,970,0,132.30,-42.30,0.0
2021-01-01 04:30:00,0,0,-2.3,0.13,3.5,51.19,7.1,971,0,119.94,-29.94,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:30:00,0,0,2.5,0.13,2.9,62.78,9.2,969,0,121.52,-31.52,0.0
2021-12-31 20:30:00,0,0,2.4,0.13,3.6,64.60,8.7,969,0,133.88,-43.88,0.0
2021-12-31 21:30:00,0,0,2.1,0.13,3.9,65.10,8.3,969,0,146.24,-56.24,0.0


In [16]:
# so a 1kW system will produce 1kWh of DC for 1 PSH (idealized)

# inverter efficiency is around 95%

# system losses are around 14%

# set the system size (recall that we assume 80amp AC charger (19 kW), so we need roughly 25 kW of DC solar capacity in the best case)
dc_system_size = 25.0 # kW

INVERTER_EFFICIENCY = 0.95
SYSTEM_LOSS = 0.14

df['Solar Generation (kWh)'] = dc_system_size * (df['GTI'] / 1000) * (1-SYSTEM_LOSS) * INVERTER_EFFICIENCY

display(df)

,DHI,DNI,Dew Point,Surface Albedo,Wind Speed,Relative Humidity,Temperature,Pressure,GHI,Solar Zenith Angle,Solar Elevation Angle,GTI,Solar Generation (kWh)
datetime,,,,,,,,,,,,,
2021-01-01 00:30:00,0,0,-0.8,0.13,3.7,55.86,7.5,969,0,166.59,-76.59,0.0,0.0
2021-01-01 01:30:00,0,0,-1.6,0.13,3.4,53.24,7.3,970,0,156.65,-66.65,0.0,0.0
2021-01-01 02:30:00,0,0,-2.0,0.13,3.4,52.12,7.2,970,0,144.67,-54.67,0.0,0.0
2021-01-01 03:30:00,0,0,-2.2,0.13,3.4,51.76,7.1,970,0,132.30,-42.30,0.0,0.0
2021-01-01 04:30:00,0,0,-2.3,0.13,3.5,51.19,7.1,971,0,119.94,-29.94,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:30:00,0,0,2.5,0.13,2.9,62.78,9.2,969,0,121.52,-31.52,0.0,0.0
2021-12-31 20:30:00,0,0,2.4,0.13,3.6,64.60,8.7,969,0,133.88,-43.88,0.0,0.0
2021-12-31 21:30:00,0,0,2.1,0.13,3.9,65.10,8.3,969,0,146.24,-56.24,0.0,0.0


In [17]:
print(df['Solar Generation (kWh)'].tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3395609565998898, 5.043197908473972, 8.395618313268843, 10.828231673356086, 12.06967182510165, 11.907134200311278, 10.511553682884893, 7.928850599516135, 4.477640085007311, 0.697017878624229, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.663859571621255, 4.649056458276777, 7.935463383736511, 10.36577735831095, 11.624645003041522, 10.544116714326037, 9.624949059113149, 6.794756320591414, 2.985512944494681, 0.5040921231183768, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5719000000000001, 4.428142641138144, 7.6327621085285795, 10.112599652691001, 11.30414515738402, 10.396171950878458, 7.763046712723405, 6.941458455394999, 3.470166811567313, 0.6487690223499784, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.166337200795028, 4.630919925224831, 7.915268867830716, 10.378389369135622, 11.613043442550746, 11.554358957823627, 10.154546245248923, 7.667883227968506, 4.35982300793074